# 02 - Entrainement du CycleGAN

Ce notebook entraine un CycleGAN pour apprendre la transformation
bidirectionnelle entre images agricoles normales et en conditions de secheresse.

**Pipeline :**
1. Preparer les donnees (domaine A = normal, domaine B = secheresse simulee)
2. Initialiser le CycleGAN
3. Entrainer (~3-5h sur GPU T4 Colab)
4. Visualiser les resultats
5. Sauvegarder le modele

In [ ]:
# Setup (decommenter sur Colab)
# !git clone https://github.com/aymenssf/SatelliteGAN-Climate-Agriculture.git
# %cd SatelliteGAN-Climate-Agriculture
# !pip install -q -r requirements.txt

import sys
import os
sys.path.insert(0, os.path.join(os.getcwd()))

import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset

from src.config import DEVICE, CYCLEGAN, IMAGE_SIZE
from src.dataset import get_agricultural_dataset, split_dataset
from src.preprocessing import (
    get_cyclegan_transform, get_eval_transform,
    simulate_drought, denormalize, tensor_to_numpy
)
from src.cyclegan.train import CycleGANTrainer
from src.evaluation.visualization import (
    show_cyclegan_results, show_comparison, plot_training_losses
)

print(f"Device : {DEVICE}")
print(f"Configuration CycleGAN : {CYCLEGAN}")

## 1. Preparation des donnees

On cree deux domaines :
- **Domaine A** : images agricoles normales (EuroSAT)
- **Domaine B** : images agricoles avec secheresse simulee

Le CycleGAN n'a pas besoin de paires appariees, mais on cree
les deux domaines a partir du meme dataset pour comparer.

In [ ]:
from torchvision import transforms
from PIL import Image

# Charger le dataset brut (sans normalisation) pour appliquer la simulation
raw_transform = transforms.Resize(IMAGE_SIZE)
raw_dataset = get_agricultural_dataset(transform=raw_transform)

# Split
train_set, val_set, _ = split_dataset(raw_dataset)

print(f"Images d'entrainement : {len(train_set)}")

In [ ]:
# Creer les tensors pour les deux domaines
cyclegan_transform = get_cyclegan_transform()
eval_transform = get_eval_transform()

def prepare_paired_batch(dataset, transform, n_max=None):
    """
    Prepare les batches pour le CycleGAN.
    Domaine A = images normales normalisees
    Domaine B = images secheresse normalisees
    """
    images_a = []
    images_b = []
    n = min(len(dataset), n_max) if n_max else len(dataset)

    for i in range(n):
        img, _ = dataset[i]

        # Convertir en PIL si c'est un tensor
        if isinstance(img, torch.Tensor):
            img_pil = transforms.ToPILImage()(img)
        else:
            img_pil = img

        # Domaine A : normal
        img_a = transform(img_pil)

        # Domaine B : secheresse simulee
        drought_pil = simulate_drought(img_pil, severity=0.6)
        img_b = transform(drought_pil)

        images_a.append(img_a)
        images_b.append(img_b)

    return torch.stack(images_a), torch.stack(images_b)

print("Preparation des donnees d'entrainement...")
train_a, train_b = prepare_paired_batch(train_set, cyclegan_transform)
print(f"Domaine A : {train_a.shape}")
print(f"Domaine B : {train_b.shape}")

In [ ]:
# Creer le DataLoader
train_dataset = TensorDataset(train_a, train_b)
train_loader = DataLoader(
    train_dataset,
    batch_size=CYCLEGAN['batch_size'],
    shuffle=True,
    drop_last=True
)

print(f"Nombre de batches par epoch : {len(train_loader)}")

## 2. Initialisation du CycleGAN

In [ ]:
# Creer le trainer
trainer = CycleGANTrainer()

# Compter les parametres
n_params_G = sum(p.numel() for p in trainer.G_A2B.parameters())
n_params_D = sum(p.numel() for p in trainer.D_A.parameters())
print(f"Parametres Generateur : {n_params_G:,}")
print(f"Parametres Discriminateur : {n_params_D:,}")
print(f"Total : {2*n_params_G + 2*n_params_D:,}")

## 3. Entrainement

L'entrainement alterne entre les generateurs et les discriminateurs.
Le replay buffer stabilise l'entrainement en evitant que le discriminateur
ne s'adapte uniquement aux dernieres images generees.

In [ ]:
# Entrainer le modele
# Reduire n_epochs pour un test rapide (ex: 5-10 epochs)
# Pour l'entrainement complet, utiliser CYCLEGAN['n_epochs'] (100)
N_EPOCHS = CYCLEGAN['n_epochs']  # mettre 5 pour un test rapide

history = trainer.train(train_loader, n_epochs=N_EPOCHS)

## 4. Courbes de perte

In [ ]:
# Afficher les courbes de perte
plot_training_losses(history, title='CycleGAN - Courbes de perte')

## 5. Visualisation des resultats

In [ ]:
# Preparer quelques images de validation
print("Preparation des donnees de validation...")
val_a, val_b = prepare_paired_batch(val_set, eval_transform, n_max=8)

# Generer les transformations
with torch.no_grad():
    val_a_dev = val_a.to(DEVICE)
    val_b_dev = val_b.to(DEVICE)

    fake_b = trainer.G_A2B(val_a_dev)    # Normal -> Secheresse
    fake_a = trainer.G_B2A(val_b_dev)    # Secheresse -> Normal
    cycle_a = trainer.G_B2A(fake_b)      # Normal -> Secheresse -> Normal
    cycle_b = trainer.G_A2B(fake_a)      # Secheresse -> Normal -> Secheresse

# Afficher les resultats complets
show_cyclegan_results(
    val_a_dev.cpu(), fake_b.cpu(), cycle_a.cpu(),
    val_b_dev.cpu(), fake_a.cpu(), cycle_b.cpu(),
    n_samples=4
)

In [ ]:
# Comparaison directe Normal vs CycleGAN secheresse
show_comparison(
    val_a.cpu(), fake_b.cpu(),
    n_samples=6,
    labels=('Normal (reel)', 'Secheresse (CycleGAN)'),
    title='Transformation Normal -> Secheresse par CycleGAN'
)

## 6. Resume

**Observations :**
- Le CycleGAN apprend a transformer les images normales en images seches
- La coherence cyclique preserve la structure spatiale (routes, limites de parcelles)
- La perte d'identite aide a preserver les teintes de couleur

**Prochaine etape :** Entrainer le modele de diffusion (DDPM) sur les images
de secheresse pour generer de nouveaux echantillons synthetiques.